Les pb :
- comment encoder le fichier test vu qu'on a pas le prix associé 
- comment appliquer ca sur un kfold (d'abord est ce utile?) ? du coup valeurs de chaque var vont etre différentes à chaque fois ?

In [2]:
import pandas as pd
import numpy as np
from sklearn import base
from sklearn.model_selection import KFold


In [3]:
data = pd.read_csv('new_result_requetes.csv')

data["city"]=pd.Categorical(data["city"],ordered=False)
data["language"]=pd.Categorical(data["language"],ordered=False)
data["group"]=pd.Categorical(data["group"],ordered=False)
data["brand"]=pd.Categorical(data["brand"],ordered=False)

data["mobile"]=pd.Categorical(data["mobile"],ordered=False)
data["parking"]=pd.Categorical(data["parking"],ordered=False)
data["pool"]=pd.Categorical(data["pool"],ordered=False)
data["children_policy"]=pd.Categorical(data["children_policy"],ordered=False)

data["hotel_id"]=pd.Categorical(data["hotel_id"],ordered=False)
data["avatar_id"]=pd.Categorical(data["avatar_id"],ordered=False)

data["price"]=pd.DataFrame(data["price"], dtype=float)

In [9]:
data.columns

Index(['order_requests', 'nb_requete', 'hotel_id', 'price', 'stock', 'city',
       'date', 'language', 'mobile', 'avatar_id', 'group', 'brand', 'parking',
       'pool', 'children_policy'],
      dtype='object')

## Smoothing with target encoding

In [1]:
def calc_smooth_mean(df, by, on, m):
    # Compute the global mean
    mean = df[on].mean()

    # Compute the number of values and the mean of each group
    agg = df.groupby(by)[on].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + m * mean) / (counts + m)

    # Replace each value by the according smoothed mean
    return df[by].map(smooth)

In [4]:
C = calc_smooth_mean(data, by='city', on='price', m=10)
#df['x_1'] = calc_smooth_mean(df, by='x_1', on='y', m=10)

In [27]:
new_dataset = pd.DataFrame(columns = ['order_requests','nb_requete', 'hotel_id', 'price', 'stock', 'city',
                                      'date', 'language', 'group', 'brand'])

In [28]:
new_dataset['price'] = data['price']
new_dataset['nb_requete'] = data['nb_requete']
new_dataset['stock'] = data['stock']
new_dataset['date'] = data['date']
new_dataset['order_requests'] = data['order_requests']

In [32]:
new_dataset['hotel_id'] = calc_smooth_mean(data, by='hotel_id', on='price', m=300)
new_dataset['city'] = calc_smooth_mean(data, by='city', on='price', m=300)
new_dataset['language'] = calc_smooth_mean(data, by='language', on='price', m=300)
#new_dataset['mobile'] = calc_smooth_mean(data, by='mobile', on='price', m=300)
new_dataset['group'] = calc_smooth_mean(data, by='group', on='price', m=300)
new_dataset['brand'] = calc_smooth_mean(data, by='brand', on='price', m=300)
#new_dataset['parking'] = calc_smooth_mean(data, by='parking', on='price', m=300)
#new_dataset['pool'] = calc_smooth_mean(data, by='pool', on='price', m=300)
#new_dataset['children_policy'] = calc_smooth_mean(data, by='children_policy', on='price', m=300)

In [29]:
from sklearn.preprocessing import OneHotEncoder
X_hot = data[["mobile","parking","pool","children_policy"]]
enc = OneHotEncoder()
enc.fit(X_hot)
Xenc=enc.transform(X_hot).toarray() #enc.fit_transform???
dataDum=pd.DataFrame(Xenc, columns=enc.get_feature_names_out())
dataDum

,mobile_0,mobile_1,parking_0,parking_1,pool_0,pool_1,children_policy_0,children_policy_1,children_policy_2
0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
3,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
4,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
422193,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
422194,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
422195,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
422196,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0


In [30]:
new_dataset = pd.concat([new_dataset,dataDum],axis=1)

In [36]:
new_dataset.head()

,order_requests,nb_requete,hotel_id,price,stock,city,date,language,group,brand,mobile_0,mobile_1,parking_0,parking_1,pool_0,pool_1,children_policy_0,children_policy_1,children_policy_2
0,1,1,145.987310,134.0,58,147.466978,30,164.770112,96.742935,96.742935,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,1,1,185.661381,155.0,97,147.466978,30,164.770112,151.909052,206.391349,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
2,1,1,128.835561,94.0,21,147.466978,30,164.770112,96.742935,96.742935,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
3,1,1,170.692270,158.0,74,147.466978,30,164.770112,228.502094,201.690180,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
4,1,1,106.886787,73.0,21,147.466978,30,164.770112,151.909052,77.842873,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0


In [17]:
data.head()

,order_requests,nb_requete,hotel_id,price,stock,city,date,language,mobile,avatar_id,group,brand,parking,pool,children_policy
0,1,1,859,134.0,58,valletta,30,portuguese,0,65736,Independant,Independant,0,0,0
1,1,1,719,155.0,97,valletta,30,portuguese,0,65736,Yin Yang,Royal Lotus,0,0,0
2,1,1,160,94.0,21,valletta,30,portuguese,0,65736,Independant,Independant,1,0,0
3,1,1,504,158.0,74,valletta,30,portuguese,0,65736,Morriott International,Morriot,0,0,0
4,1,1,295,73.0,21,valletta,30,portuguese,0,65736,Yin Yang,8 Premium,0,0,0


In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    new_dataset.drop(columns = ['price']),
    new_dataset['price'],
    test_size=0.2,
    random_state=42,
)
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

In [39]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import QuantileTransformer, StandardScaler

from sklearn.ensemble import RandomForestRegressor


rf = make_pipeline(
    StandardScaler(),
    RandomForestRegressor(max_depth=20),
)

#rf = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=20)
rf.fit(X_train, y_train)
pred = rf.predict(X_test)

In [40]:
print(f'with target encoding {rf.score(X_test,y_test, pred):.4f}')

with target encoding 0.9986


### Tester sur le fichier test officiel mais comment encoder?


In [41]:
test = pd.read_csv('./defi-ia-2023/test_set3.csv')
test.head()

,nb_requete,index,order_requests,city,date,language,mobile,avatar_id,hotel_id,stock,group,brand,parking,pool,children_policy
0,1,0,1,vilnius,21,romanian,0,1,161,46,Boss Western,J.Halliday Inn,1,0,0
1,1,1,1,vilnius,21,romanian,0,1,187,32,Accar Hotels,Marcure,1,1,0
2,1,2,1,vilnius,21,romanian,0,1,279,12,Independant,Independant,1,0,0
3,1,3,1,vilnius,21,romanian,0,1,395,10,Accar Hotels,Ibas,0,0,0
4,1,4,1,vilnius,21,romanian,0,1,488,42,Accar Hotels,Safitel,1,1,1


In [42]:
test["city"]=pd.Categorical(test["city"],ordered=False)
test["language"]=pd.Categorical(test["language"],ordered=False)
test["group"]=pd.Categorical(test["group"],ordered=False)
test["brand"]=pd.Categorical(test["brand"],ordered=False)

test["mobile"]=pd.Categorical(test["mobile"],ordered=False)
test["parking"]=pd.Categorical(test["parking"],ordered=False)
test["pool"]=pd.Categorical(test["pool"],ordered=False)
test["children_policy"]=pd.Categorical(test["children_policy"],ordered=False)

test["hotel_id"]=pd.Categorical(test["hotel_id"],ordered=False)
test["avatar_id"]=pd.Categorical(test["avatar_id"],ordered=False)

In [45]:
from category_encoders import TargetEncoder

X = test.drop(columns=["stock","date","nb_requete","order_requests",'avatar_id','parking','pool'])
enc = TargetEncoder()
#enc.fit(X,y)

Xenc=enc.fit(X)

dataDum=pd.DataFrame(Xenc, columns= enc.get_feature_names())

dataQuant = df[["date","nb_requete","stock"]]

dfC = pd.concat([dataQuant,dataDum],axis=1)
dfC.head()

C:\Users\PC\anaconda3\envs\ProjetIA\lib\site-packages\category_encoders\target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
C:\Users\PC\anaconda3\envs\ProjetIA\lib\site-packages\category_encoders\target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


ValueError: Supervised encoders need a target for the fitting. The target cannot be None